In [261]:
import networkx as nx
from copy import deepcopy
from operator import itemgetter
import matplotlib.pyplot as plt
%matplotlib inline

In [262]:
def check_completeness(solution, variables):
    return True if len(solution) == len(variables) else False

In [263]:
def static_variable_selection(solution, csp, debug=True):
    unassigned_variables = [x for x in csp[0] if x not in solution]
    variable = unassigned_variables[0]
    if debug: print("Selected variable:"+ str(variable))
    return variable

In [264]:
def static_domain_selection(variable, assignment, csp, debug=True):
    for domain_value in csp[1][variable]:
        if debug: print("Selected value" + str(domain_value) + "for" + str(variable) )
        yield domain_value

In [265]:
def check_value_consistency(variable, value, assignment, constraints, debug=True):
    if assignment is None: return False
    for v in constraints:
        if v in assignment:
            if value is assignment[v]:
                if debug: print(str(variable)+":"+ str(value)+ "conflicts with"+str(v)+" :" + str(assignment[v]))
                return False
    return True

In [266]:
def no_inference(csp, variable, value, assignment, debug=True):
    return True

In [267]:
def backtrack(assignment, csp, params, debug=True):
    select_unassigned_variable, order_domain_values, inference = params
    if debug: print("Solution: " +str(assignment))
    if check_completeness(assignment, csp[0]): return True, assignment
    variable = select_unassigned_variable(assignment, csp, debug=debug)
    for value in order_domain_values(variable, assignment, csp, debug):
        if check_value_consistency(variable, value, assignment, csp[2][variable], debug=debug):
            assignment[variable], m_csp = value, deepcopy(csp)
            if inference(csp, variable, value, assignment, debug=debug):
                result, assignment = backtrack(deepcopy(assignment), csp, params, debug=debug)
                if result: return True, assignment
            del assignment[variable]
            csp = m_csp
    return False, assignment

In [268]:
def backtracking_search(csp, params, debug=True):
    solution = dict()
    return backtrack(solution, csp, params, debug=debug)

In [269]:
def get_adjacency_list(nodes, edges):
    adjacency_list = dict()
    for node in nodes:
        aList = set()
        [aList.add(x) for x, y in edges if x != node and y == node]
        [aList.add(y) for x, y in edges if y != node and x == node]
        adjacency_list[node] = aList
    return adjacency_list

In [270]:
def minimum_remaining_values():
    pass
def forward_checking():
    pass
def least_contraining_value():
    pass


In [271]:
def color_map(world, colors=["blue", "green", "red", "yellow"], params=(minimum_remaining_values, least_contraining_value, forward_checking), debug=True):
    variables = world[0]
    domains = dict([(x, colors) for x in variables])
    constraints = get_adjacency_list(world[0], world[1])
    result, solution = backtracking_search((variables, domains, constraints), params, debug=debug)
    if not result: print("No Solution")                  
    else: return solution

In [272]:
def draw_color_map(world, colors, pos=None):
    if colors is None: return
    fig = plt.figure()
    G = nx.Graph()
    if pos is None:
        pos = nx.spring_layout(G)
    G.add_nodes_from(world[0])
    G.add_edges_from(world[1])
    node_colors = [colors[x] for x in G.nodes()]
    nx.draw(G, node_color=node_colors, node_size=700, alpha=0.5)
    plt.axis('off')
    fig.set_size_inches(8, 8)
    plt.show()

In [273]:
def connecticut_map():
    nodes = ["fairfield", "litchfield", "new haven", "hartford", "middlesex", "tolland", "new london", "windham"]
    edges = [("fairfield", "litchfield"), ("fairfield", "new haven"),
             ("litchfield", "new haven"), ("litchfield", "hartford"),
             ("new haven", "hartford"), ("new haven", "middlesex"),
             ("hartford", "middlesex"), ("hartford", "tolland"), ("hartford", "new london"),
             ("middlesex", "new london"),
             ("tolland", "new london"), ("tolland", "windham"),
             ("new london", "windham")]
    return (nodes, edges)

In [274]:
world = connecticut_map()
solution = color_map(world, params=(static_variable_selection, static_domain_selection, no_inference))
#draw_color_map(world, solution)

Solution: {}
Selected variable:fairfield
Selected valueblueforfairfield
Selected valuegreenforfairfield
Selected valueredforfairfield
Selected valueyellowforfairfield
No Solution


In [275]:
def minimum_remaining_values(solution, csp, debug=True):
    unassigned_variables = [x for x in csp[0] if x not in solution]
    variable_domain_count = [(x, len(csp[1][x])) for x in unassigned_variables]
    variables = sorted(variable_domain_count, key=itemgetter(1))
    if debug: print(" Selected variable " +str(variables[0][0])+ " with value count " + str(variables[0][1]))
    return variables[0][0]

In [276]:
def forward_checking(csp, variable, value, assignment, debug=True):
    unassigned_neighbors = [y for y in csp[2][variable] if y not in assignment]
    for x in unassigned_neighbors:
        if value in csp[1][x]:
            if debug: print(" Removing " +str(value) + " from " +str(x))
            csp[1][x] = [c for c in csp[1][x] if c is not value]
        if len(csp[1][x]) == 0: return False
    return True

In [277]:
world = connecticut_map()
solution = color_map(world, params=(minimum_remaining_values, static_domain_selection, forward_checking))
#draw_color_map(world, solution)

Solution: {}
 Selected variable fairfield with value count 4
Selected valueblueforfairfield
 Removing blue from litchfield
 Removing blue from new haven
Solution: {'fairfield': 'blue'}
 Selected variable litchfield with value count 3
Selected valuegreenforlitchfield
 Removing green from hartford
 Removing green from new haven
Solution: {'fairfield': 'blue', 'litchfield': 'green'}
 Selected variable new haven with value count 2
Selected valueredfornew haven
 Removing red from hartford
 Removing red from middlesex
Solution: {'fairfield': 'blue', 'litchfield': 'green', 'new haven': 'red'}
 Selected variable hartford with value count 2
Selected valueblueforhartford
 Removing blue from new london
 Removing blue from tolland
 Removing blue from middlesex
Solution: {'fairfield': 'blue', 'litchfield': 'green', 'new haven': 'red', 'hartford': 'blue'}
 Selected variable middlesex with value count 2
Selected valuegreenformiddlesex
 Removing green from new london
Solution: {'fairfield': 'blue', 'l

In [278]:
def least_contraining_value(variable, assignment, csp, debug=True):
    neighbors = csp[2][variable]
    value_count = []
    for value in csp[1][variable]:
        count = 0
        for neighbor in neighbors:
            if value in csp[1][neighbor]:
                count = count + 1
        value_count.append((value, count))
    counted = sorted(value_count, key=itemgetter(1), reverse=True)
    for value, count in counted:
        if debug: print("  Selected value " + str(value) + "for " + str(variable) )
        yield value

In [279]:
def degree_heuristic(solution, csp, debug=True):
    unassigned_variables = [x for x in csp[0] if x not in solution]
    variable_domain_count = [(x, len(csp[2][x])) for x in unassigned_variables]
    variables = sorted(variable_domain_count, key=itemgetter(1), reverse=True)
    if debug: print ("  Selected variable "+ str(variables[0][0])+ " with constraint count " + str(variables[0][1]))
    return variables[0][0]

In [280]:
world = connecticut_map()
solution = color_map(world, params=(degree_heuristic, least_contraining_value, forward_checking),)
#draw_color_map(world, solution)

Solution: {}
  Selected variable hartford with constraint count 5
  Selected value bluefor hartford
 Removing blue from new london
 Removing blue from new haven
 Removing blue from litchfield
 Removing blue from tolland
 Removing blue from middlesex
Solution: {'hartford': 'blue'}
  Selected variable new haven with constraint count 4
  Selected value greenfor new haven
 Removing green from fairfield
 Removing green from litchfield
 Removing green from middlesex
Solution: {'hartford': 'blue', 'new haven': 'green'}
  Selected variable new london with constraint count 4
  Selected value redfor new london
 Removing red from windham
 Removing red from middlesex
 Removing red from tolland
Solution: {'hartford': 'blue', 'new haven': 'green', 'new london': 'red'}
  Selected variable litchfield with constraint count 3
  Selected value redfor litchfield
 Removing red from fairfield
Solution: {'hartford': 'blue', 'new haven': 'green', 'new london': 'red', 'litchfield': 'red'}
  Selected variable m